In [ ]:
# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/workshops_preparation/data/mimic_pulseOx_data.csv")

In [ ]:
import numpy as np

# To prevent data leakage, we're setting lab results or clinical status with a timedelta of zero or more to NaN.
timedelta_cols = [col for col in df.columns if col.startswith("delta_")]
timedelta_cols_dict = {}
for col in timedelta_cols:
  if col.split("delta_")[-1] in df.columns:
    timedelta_cols_dict[col] = [col.split("delta_")[-1]]

timedelta_cols_dict["delta_vent_start"] = ["ventilation_status", "invasive_vent"]
timedelta_cols_dict["delta_vp_start"] = ["norepinephrine_equivalent_dose"]

# Replace with nan
for timedelta_col, corresponding_col in timedelta_cols_dict.items():
  for col in corresponding_col:
    df[[timedelta_col, col]] = df[[timedelta_col, col]].mask(df[timedelta_col]>=0, np.nan)

In [ ]:
df = df[(~df["SpO2"].isna()) & (df.SaO2 <= 100) & (df.SpO2 <= 100) & (df.SaO2 >= 65) & (df.SpO2 >= 65)]
df = df[df['delta_SpO2'] >= -10]

In [ ]:
import os

# Function to create the directory if it doesn't exist
def create_directory_if_not_exists(path):
    directory = os.path.dirname(path)
    os.makedirs(directory, exist_ok=True)

# Save function to be used for both train and test datasets
def save_dataset(dataset, path_to_save):
    create_directory_if_not_exists(path_to_save)
    dataset.to_csv(path_to_save, index=False)

In [ ]:
save_dataset(df, "/content/drive/MyDrive/workshops_preparation/for_japan/solutions/output/workshop_1_data_final.csv")